Can I predict the existence of subfields with some cool unsupervised learning algorithm? 

For starters, let's just use regular n-grams. A more advanced version would be to look for noun phrases or J&K POS tags.

In [23]:
#Need to add parent directoy to sys.path to find 'metadataDB'
import sys
sys.path.append('../../')

%matplotlib inline
# import matplotlib.pyplot as plt 
import time
import numpy as np
# import scipy as sp
import re
from collections import Counter
import itertools
import random
import pickle

# Natural language processing toolkit
# To use this, run nltk.download() and download 'stopwords'
# from nltk.corpus import stopwords
# s=stopwords.words('english') + ['']

# Machine learning
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.cluster import KMeans
from sklearn.decomposition import SparsePCA
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
# from sklearn import metrics
from sklearn.externals import joblib

# SQL
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from metadataDB.declareDatabase import *
from sqlalchemy import or_, and_

engine = create_engine("sqlite:///../../arXiv_metadata.db", echo=False)
Base.metadata.bind = engine
DBsession = sessionmaker(bind=engine)
session = DBsession()

In [6]:
# What are the available categories?
categories = sorted([x.name for x in session.query(Category)])
print categories 

[u'acc-phys', u'adap-org', u'alg-geom', u'ao-sci', u'astro-ph', u'astro-ph.CO', u'astro-ph.EP', u'astro-ph.GA', u'astro-ph.HE', u'astro-ph.IM', u'astro-ph.SR', u'atom-ph', u'bayes-an', u'chao-dyn', u'chem-ph', u'comp-gas', u'cond-mat', u'cond-mat.dis-nn', u'cond-mat.mes-hall', u'cond-mat.mtrl-sci', u'cond-mat.other', u'cond-mat.quant-gas', u'cond-mat.soft', u'cond-mat.stat-mech', u'cond-mat.str-el', u'cond-mat.supr-con', u'cs.AI', u'cs.AR', u'cs.CC', u'cs.CE', u'cs.CG', u'cs.CL', u'cs.CR', u'cs.CV', u'cs.CY', u'cs.DB', u'cs.DC', u'cs.DL', u'cs.DM', u'cs.DS', u'cs.ET', u'cs.FL', u'cs.GL', u'cs.GR', u'cs.GT', u'cs.HC', u'cs.IR', u'cs.IT', u'cs.LG', u'cs.LO', u'cs.MA', u'cs.MM', u'cs.MS', u'cs.NA', u'cs.NE', u'cs.NI', u'cs.OH', u'cs.PF', u'cs.PL', u'cs.RO', u'cs.SC', u'cs.SD', u'cs.SE', u'cs.SI', u'cs.SY', u'dg-ga', u'funct-an', u'gr-qc', u'hep-ex', u'hep-lat', u'hep-ph', u'hep-th', u'math-ph', u'math.AC', u'math.AG', u'math.AP', u'math.AT', u'math.CA', u'math.CO', u'math.CT', u'math.CV',

In [7]:
abstract_all_tmp = {'category': [], 'abstract': []}
# category_list = sorted(['atom-ph', 'quant-ph', 'optics', 'nlin', 'str-el', 'stat'])
category_list = sorted(['quant-ph', 'str-el', 'hep-', 'mtrl-sci', 'supr-con'])
category_len = len(category_list)

start = time.time()
for item in category_list:
    query = session.query(Article_Category)\
                        .join(Category)\
                        .join(Article)\
                        .filter(Category.name.like('%' + item + '%'))
#     query = session.query(Article_Category)\
#                         .join(Category)\
#                         .join(Article)\
#                         .filter(Category.name.like('%' + item + '%'))
    result = [' '.join(x.article.abstract.split()) for x in query]
    abstract_all_tmp['abstract'].extend(result)
    abstract_all_tmp['category'].extend([item]*len(result))
print time.time() - start
# for item in query:
#     abstract_all['category'].append(item.category.name)
#     abstract_all['abstract'].append(' '.join(item.article.abstract.split()))
# print time.time() - start
# abstract_all['atom-ph'] = [x.article.abstract for x in query.all()]
# session.close_all()

435.189222097


In [8]:
# Breakdown of categories?
count = Counter(abstract_all_tmp['category'])
for key, val in count.iteritems():
    print '{:<15}{}'.format(key, val)
print '{:<15}{}'.format('Total', len(abstract_all_tmp['abstract']))

supr-con       24002
str-el         35651
hep-           217391
mtrl-sci       38442
quant-ph       60594
Total          376080


In [9]:
##Oops! How many overlapping articles do we have? I forgot that arXiv categories aren't unique.
# Let's remove all duplicates.
# This is slow but I am tired.

counter_duplicate = Counter(abstract_all_tmp['abstract'])

abstract_all = {'category': [], 'abstract': []}
for cat, abstract in itertools.izip(abstract_all_tmp['category'], abstract_all_tmp['abstract']):
    if counter_duplicate[abstract] == 1:
        abstract_all['category'].append(cat)
        abstract_all['abstract'].append(abstract)
print len(abstract_all['category'])
print len(abstract_all['abstract'])

261661
261661


In [10]:
# Breakdown of categories? That's a lot of repetition!!!
count = Counter(abstract_all['category'])
for key, val in count.iteritems():
    print '{:<15}{}'.format(key, val)
print '{:<15}{}'.format('Total', len(abstract_all['abstract']))

supr-con       13400
str-el         20290
hep-           144269
mtrl-sci       31617
quant-ph       52085
Total          261661


In [11]:
# Train on 80% of the data. Random_state ensures that we always get the same result.
x_train, x_test, y_train, y_test = train_test_split(abstract_all['abstract'],
                                                    abstract_all['category'],
                                                    random_state=42,
                                                    train_size=0.8)

counter_train = Counter(y_train)

Okay, I lied, I'm starting with supervised learning (as a comparison). We're looking at ~60-70% accuracy for these cateogories.

In [12]:
#SVC(kernel='linear') is good
clf_supervised = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))),
                           ('tfidf', TfidfTransformer()),
#                            ('clf', LinearSVC())])
                           ('clf', LinearSVC(C=1,penalty='l1',dual=False,))])
start = time.time()
clf_supervised.fit(x_train, y_train)
print time.time() - start

start = time.time()
predict = clf_supervised.predict(x_test)
print time.time() - start
#print text_abstract_clf.predict(train_abstract)

804.721962929
48.4372420311


In [13]:
print(classification_report(y_test, predict))
print('Accuracy score: %0.2f' % accuracy_score(y_test, predict))

             precision    recall  f1-score   support

       hep-       0.98      0.98      0.98     28613
   mtrl-sci       0.88      0.91      0.90      6309
   quant-ph       0.93      0.92      0.92     10547
     str-el       0.85      0.80      0.82      4182
   supr-con       0.90      0.88      0.89      2682

avg / total       0.94      0.94      0.94     52333

Accuracy score: 0.94


Find the most important words.

In [14]:
# Most important chunks. See http://scikit-learn.org/stable/auto_examples/text/document_clustering.html
most_important_words = clf_supervised.named_steps['clf'].coef_.argsort()[:, ::-1]

terms =  clf_supervised.named_steps['vect'].get_feature_names()
for i in range(len(category_list)):
    print "Category %s:" % (category_list[i])
    print ', '.join([terms[x] for x in most_important_words[i, :20]])
    print ''

Category hep-:
qcd, quark, holographic, hadron, neutrino, hep, gev, lhc, meson, ads, quarks, cosmological, brane, yang mills, branes, dark matter, mesons, hadronic, quantum group, parton

Category mtrl-sci:
lennard, fracture, phase field, molecular dynamics, colloidal, relaxor, exact exchange, paraelectric, first principles, ff state, spin transfer, nw, elasticity, packing, density functional, ferrite, acid, materials, solids, micromagnetic

Category quant-ph:
quantum, quantum walks, bell, bohmian, transmon, nitrogen vacancy, optomechanical, photosynthetic, entanglement, exceptional points, quant, fluctuation electromagnetic, bose hubbard, pseudospin symmetry, compton wavelength, quantum annealing, plasma model, toric code, telecom, completely positive

Category str-el:
dynamical mean field, test our theory, falicov, kondo, hidden order, quantum monte, quantum critical, hubbard, electron glasses, holstein, wigner crystal, heavy fermion, numerical renormalization group, 5f, double excha

Now, try KMeans clustering. 
See: http://scikit-learn.org/stable/auto_examples/text/document_clustering.html

In [17]:
n_clusters = 10
# Reduce n_init to 10 for testing purposes.
clf_unsupervised = Pipeline([('vect', CountVectorizer(ngram_range=(1,3), stop_words='english')),
                             ('tfidf', TfidfTransformer()),
                             ('clf', KMeans(n_clusters=n_clusters, n_init=10))])
start = time.time()
clf_unsupervised.fit(x_train)
print time.time() - start

start = time.time()
predict_train = clf_unsupervised.predict(x_train)
predict = clf_unsupervised.predict(x_test)
print time.time() - start


10316.5555911
127.936686993


In [18]:
# Which clusters most closely align with which the original categories?
# Find the strongest correlation, and assign that cluster to the category. Iterate.
# matrix_train = [[sum((a==cat and b==x for a,b in zip(y_train, predict_train)))
#                    for x in range(0,n_clusters)] 
#                    for cat in category_list]


counter_category = Counter(y_train)
counter_cluster = Counter(predict_train)
accuracy_train_initial = np.array(
#     [[sum((a==cat and b==x for a,b in zip(y_train, predict_train)))*1./counter_cluster[x]
    [[sum((a==cat and b==x for a,b in zip(y_train, predict_train)))*1./counter_category[cat]
           for x in range(0,n_clusters)] 
           for cat in category_list])
clusterToCategory = dict()
# categoryToCluster = dict()

for cluster, item in enumerate(np.argmax(accuracy_train_initial, axis=0).tolist()):
    clusterToCategory[cluster] = category_list[item]


# category_list_remaining = list(category_list)
# cluster_list_remaining = range(0, n_clusters)
# for x in range(0, len(category_list)):
#     accuracy_train = np.array(
#                         [[sum((a==cat and b==x for a,b in zip(y_train, predict_train)))*1./counter_cluster[x]
#                            for x in cluster_list_remaining] 
#                            for cat in category_list_remaining])

    
#     # Find largest value in the category axis
#     cat_ind, cluster_ind = np.unravel_index(np.argmax(accuracy_train), accuracy_train.shape)
#     clusterToCategory[cluster_list_remaining[cluster_ind]] = category_list_remaining[cat_ind]
#     categoryToCluster[category_list_remaining[cat_ind]] = cluster_list_remaining[cluster_ind]
    
#     # Remove those entries from the lists.
#     category_list_remaining.remove(category_list_remaining[cat_ind])
#     cluster_list_remaining.remove(cluster_list_remaining[cluster_ind])
# #     break

# # The remaining clusters predict empty strings
# for item in cluster_list_remaining:
#     clusterToCategory[item] = ''
# clusterToCategory_list = sorted(clusterToCategory.values())

In [19]:
# The table is normalized by number of elements in each cluster.
print 'Training data'
print ('{:<10}' + '{:<10}'  *n_clusters).format('', *range(0, n_clusters))
print ('{:<10}' + '{:<10}'  *n_clusters).format('', *[clusterToCategory[x] for x in range(0, n_clusters)])
for cat, item in zip(category_list, accuracy_train_initial):
    print ('{:<10}' + '{:<10.2}'*n_clusters).format(cat, *item)

Training data
          0         1         2         3         4         5         6         7         8         9         
          hep-      hep-      quant-ph  str-el    hep-      hep-      quant-ph  hep-      supr-con  hep-      
hep-      0.25      0.037     0.32      0.013     0.12      0.036     0.008     0.17      0.013     0.034     
mtrl-sci  0.0017    0.00012   0.28      0.14      0.0       0.00016   0.0057    0.0004    0.57      4e-05     
quant-ph  0.01      0.00089   0.38      0.042     7.2e-05   0.00029   0.54      0.00053   0.025     0.00024   
str-el    0.012     0.00025   0.18      0.37      0.00012   0.0016    0.029     0.0       0.4       0.0       
supr-con  0.0033    9.3e-05   0.13      0.16      9.3e-05   0.0022    0.02      9.3e-05   0.68      0.0       


In [20]:
# Is there overlap between the clusters and existing categories ('ground truth')?
matrix = [[sum((a==cat and b==x for a,b in zip(y_test, predict)))
           for x in range(0,n_clusters)] 
           for cat in category_list]

print 'Test data'
print ('{:<10}' + '{:<10}'  *n_clusters).format('', *range(0, n_clusters))
print ('{:<10}' + '{:<10}'  *n_clusters).format('', *[clusterToCategory[x] for x in range(0, n_clusters)])
for cat, item in zip(category_list, matrix):
    print ('{:<10}' + '{:<10}'*n_clusters).format(cat, *item)
    
    
# # Oops, this is the same as the confusion matrix?
# tmp_reverse_category = dict([(y,x) for x,y in enumerate(category_list)])
# y_test_num = [tmp_reverse_category[x] for x in y_test]
# print ''
# print 'Confusion matrix:'
# print confusion_matrix(y_test_num, predict)

Test data
          0         1         2         3         4         5         6         7         8         9         
          hep-      hep-      quant-ph  str-el    hep-      hep-      quant-ph  hep-      supr-con  hep-      
hep-      8083      1243      5824      504       3965      1245      362       5732      479       1176      
mtrl-sci  26        0         1042      1032      0         4         51        4         4150      0         
quant-ph  186       18        2789      528       5         6         6601      20        387       7         
str-el    61        3         458       1845      0         9         182       0         1624      0         
supr-con  12        2         170       580       0         6         75        0         1836      1         


In [21]:
# We can now make a prediction based on these categories.
predict_category = [clusterToCategory[y] for y in predict]

print(classification_report(y_test, predict_category))
print('Accuracy score: %0.2f' % accuracy_score(y_test, predict_category))
print confusion_matrix(y_test, predict_category)

             precision    recall  f1-score   support

       hep-       0.98      0.75      0.85     28613
   mtrl-sci       0.00      0.00      0.00      6309
   quant-ph       0.53      0.89      0.67     10547
     str-el       0.41      0.44      0.43      4182
   supr-con       0.22      0.68      0.33      2682

avg / total       0.69      0.66      0.65     52333

Accuracy score: 0.66
[[21444     0  6186   504   479]
 [   34     0  1093  1032  4150]
 [  242     0  9390   528   387]
 [   73     0   640  1845  1624]
 [   21     0   245   580  1836]]


/Users/emarti/anaconda/lib/python2.7/site-packages/sklearn/metrics/metrics.py:1771: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [22]:
# Most important chunks. See http://scikit-learn.org/stable/auto_examples/text/document_clustering.html
order_centroids = clf_unsupervised.named_steps['clf'].cluster_centers_.argsort()[:, ::-1]

terms =  clf_unsupervised.named_steps['vect'].get_feature_names()
for i in range(n_clusters):
    print "Cluster %d (%s):" % (i, clusterToCategory[i])
    print ', '.join([terms[x] for x in order_centroids[i, :20]])
    print ''

Cluster 0 (hep-):
theory, gauge, field, theories, string, space, dimensional, non, gravity, solutions, brane, action, fields, branes, field theory, conformal, equations, scalar, algebra, model

Cluster 1 (hep-):
black, black hole, hole, black holes, holes, horizon, solutions, entropy, hawking, gravity, extremal, ads, dimensional, rotating, solution, scalar, field, theory, kerr, schwarzschild

Cluster 2 (quant-ph):
model, energy, time, results, quantum, method, using, field, equation, non, state, potential, new, present, function, order, theory, particle, study, states

Cluster 3 (str-el):
spin, magnetic, field, magnetic field, phase, state, temperature, quantum, magnetization, coupling, model, ferromagnetic, electron, order, transition, effect, spin orbit, ground, orbit, states

Cluster 4 (hep-):
neutrino, higgs, model, mass, standard model, standard, boson, cp, mixing, higgs boson, masses, mu, neutrinos, lepton, tau, decay, gev, models, supersymmetric, flavor

Cluster 5 (hep-):
pi, pi

In [ ]:
# Save model
joblib.dump(clf_supervised, 'svm_category.pkl')
joblib.dump(clf_unsupervised, 'kmeans_category.pkl')

PCA is another interesting approach.

In [ ]:
# clf_pca = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))),
#                              ('tfidf', TfidfTransformer()),
#                              ('clf', KMeans(n_components=6))])
# X = clf_pca.fit(x_train)
# # predict = clf_pca.predict(x_test)